In [2]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# names = []
# Letter_names = ["M", "S", "B", "Y"]
# for letter in Letter_names:
#     for index in range(15):
#         names.append({'file':"{0}_open_({1}).wav".format(letter, index), 'label':"{0}".format(letter)})
# file_name = pd.DataFrame(names)
# print(file_name)

In [4]:
Names = os.listdir("AllRecords//close")
Close_Names_df = pd.DataFrame(Names, columns={'Paths'})
Close_Names_df.insert(1, "Labels", len(Names) *[0])
print(Close_Names_df)

                Paths  Labels
0     a_close_(1).wav       0
1    a_close_(10).wav       0
2    a_close_(11).wav       0
3    a_close_(12).wav       0
4    a_close_(13).wav       0
..                ...     ...
211   y_close_(5).wav       0
212   y_close_(6).wav       0
213   y_close_(7).wav       0
214   y_close_(8).wav       0
215   y_close_(9).wav       0

[216 rows x 2 columns]


In [5]:
#Getting Team information
Names2 = os.listdir("AllRecords//open")
Open_Names_df = pd.DataFrame(Names2, columns={'Paths'})
Open_Names_df.insert(1, "Labels", len(Names2) *[1])
print(Open_Names_df)
all = Open_Names_df.append(Close_Names_df, ignore_index=True)
print(all)

                           Paths  Labels
0                 a_open_(1).wav       1
1                a_open_(10).wav       1
2                a_open_(11).wav       1
3                a_open_(12).wav       1
4                a_open_(13).wav       1
..                           ...     ...
214   yasmennRecording 1 (1).wav       1
215  yasmennRecording 1 (16).wav       1
216   yasmennRecording 1 (2).wav       1
217   yasmennRecording 1 (6).wav       1
218   yasmennRecording 1 (9).wav       1

[219 rows x 2 columns]
               Paths  Labels
0     a_open_(1).wav       1
1    a_open_(10).wav       1
2    a_open_(11).wav       1
3    a_open_(12).wav       1
4    a_open_(13).wav       1
..               ...     ...
430  y_close_(5).wav       0
431  y_close_(6).wav       0
432  y_close_(7).wav       0
433  y_close_(8).wav       0
434  y_close_(9).wav       0

[435 rows x 2 columns]


C:\Users\mahmo\AppData\Local\Temp\ipykernel_14016\4113759299.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = Open_Names_df.append(Close_Names_df, ignore_index=True)


In [13]:
def Extract_features(paths, whichone, dowhat = ''):
    path = os.path.join(os.path.abspath('AllRecords/{0}').format(whichone)+ ('\\') +str(paths['Paths']))
    X, sample_rate = librosa.load(path, res_type = "kaiser_fast")
    X, index    = librosa.effects.trim(X)
    chroma_stft = librosa.feature.chroma_stft(X, sr=sample_rate)
    rmse = librosa.feature.rms(y=X)
    spec_cent = librosa.feature.spectral_centroid(X, sr=sample_rate) 
    spec_bw = librosa.feature.spectral_bandwidth(X, sr=sample_rate)
    rolloff = librosa.feature.spectral_rolloff(X, sr=sample_rate)
    zcr = librosa.feature.zero_crossing_rate(X)
    mfcc = librosa.feature.mfcc(X, sr=sample_rate,n_mfcc=40) 
    return [chroma_stft, rmse, spec_cent, spec_bw, rolloff,zcr,mfcc]
    # return [chroma_stft, rmse, spec_cent, spec_bw, rolloff,zcr,mfcc]
    # return features


In [14]:
mfccs = []
mel = []
tonnetz = []
chroma = []
contrast = []
label_enter = []
DataSet = pd.DataFrame()
DataSet = all.apply(Extract_features, whichone="AllTeams", axis=1)
dd = np.array(DataSet[0])
print(DataSet)

0      [[[0.5400426, 0.9883244, 1.0, 0.99151784, 0.42...
1      [[[0.7018491, 0.46376297, 0.48847985, 0.539338...
2      [[[0.8014314, 0.976341, 1.0, 1.0, 0.67644894, ...
3      [[[0.7008797, 0.49387884, 0.5993096, 0.9219109...
4      [[[0.9034873, 0.9895297, 0.7815614, 0.71072185...
                             ...                        
430    [[[1.0, 0.82806844, 0.95903087, 1.0, 0.924533,...
431    [[[0.53762746, 0.7825138, 0.8607997, 1.0, 0.58...
432    [[[0.87527734, 0.83723205, 0.7751678, 0.787379...
433    [[[0.79039603, 0.7560549, 0.66985595, 1.0, 0.9...
434    [[[0.44455808, 0.62409323, 0.6815331, 0.826901...
Length: 435, dtype: object


C:\Users\mahmo\AppData\Local\Temp\ipykernel_14016\3386549790.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dd = np.array(DataSet[0])


In [54]:
def features_append(features):
    feature = []
    # features[0][5] = [features[0][5]]
    for i in range(len(features)):
        # print(features[0][1],'1')
        # print(features[0][2], '2')
        # print(features[0][3], '3')
        # print(features[0][4], '4')
        # print(features[0][5], '5')
        # feature.append(np.concatenate((features[i][0], features[i][1], features[i][2]
        # , features[i][3],features[i][4],features[i][5], features[i][6]), axis=0))
        to_append   = f'{np.mean(features[i][0])} {np.mean(features[i][1])} {np.mean(features[i][2])} {np.mean(features[i][3])} {np.mean(features[i][4])} {np.mean(features[i][5])}'
        for e in features[i][6]:
            to_append += f' {np.mean(e)}'
        # print(to_append.split())
        feature.append(to_append.split())
        # print(feature)

    # print("These are: ",features)
    # print("The length",len(features[0]))
    # print(feature)
    for j in range(len(features)):
        for index in range(0,len(feature[j])):
            # print(i, index)
            # print(feature[i][index])
            feature[j][index]=float(feature[j][index])

    return feature


In [56]:
# print('1st data',DataSet[2])
# print('2nd data',DataSet[0][1])
# print('3rd data',DataSet[0])
# print('4th data',DataSet[0][3])
# print('5th data',DataSet[0][4])
# print('6th data',DataSet[0][5])
# print('7th data',DataSet[0][6])
print(len(DataSet))
x = features_append(DataSet)
# x = DataSet
# df = pd.DataFrame(x)
# for i in range(len(x)):
#     x[i] = x[i][0]
# print('2nd',x[2])

435


In [57]:
X_train, X_test, y_train, y_test = train_test_split(x, all['Labels'], test_size=0.2, random_state=0)

In [58]:
clf = RandomForestClassifier(random_state=0, n_estimators=40)
clf.fit(X_train,y_train.values.ravel())

RandomForestClassifier(n_estimators=40, random_state=0)

In [59]:
y_pred = clf.predict(X_test)

In [60]:
clf.predict(X_test)

array([1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
      dtype=int64)

In [61]:
y_test

153    1
264    0
54     1
355    0
428    0
      ..
52     1
74     1
26     1
45     1
272    0
Name: Labels, Length: 87, dtype: int64

In [62]:
a = accuracy_score(y_pred, y_test)
print(a)

0.8850574712643678


In [63]:
clf.predict(x[:12])

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int64)

In [64]:
all['Labels'][:12]

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
Name: Labels, dtype: int64

In [65]:
newTest = os.listdir("AllRecords/Test/")
Test = pd.DataFrame(newTest, columns={"Paths"})
get_feat = Test.apply(Extract_features, whichone='Test', axis=1)
combine = features_append(get_feat)
# print(combine[0])
print("Length", len(combine[0]))

Length 46


In [66]:
clf.predict(combine)

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], dtype=int64)

In [67]:
pickle.dump(clf, open("Sentence-model.pkl", 'wb'))